## Creating Your First Graph and Running It in a Session

### 전체적인 실행과정
#### 그래프(플로우차트) 생성 --> 실행(Session, run) 

In [3]:
import tensorflow as tf

#### - 그래프 만들기

In [2]:
#그래프 만들기 
#노드 생성 
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")

f = x*x*y + y + 2

#### - 실행(돌리기)

In [3]:
#가장 기초적이고 기본적인 방법 1

#Session 만들기 
sess = tf.Session()

#.run :실제 실행
#.initializer : 초기화 
sess.run(x.initializer)
sess.run(y.initializer)

result = sess.run(f)
print(result)

#닫아줘야함 
sess.close()

42


In [4]:
#with 구문을 사용한 방법 2

#with 블록의 세션은 기본 세션으로 설정됨 
with tf.Session() as sess : 
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    
print(result)
    
# f.eval = tf.get_default_session().run(f)

42


In [5]:
#가장 짧은 방법 3

init = tf.global_variables_initializer() #init node를 준비 

with tf.Session() as sess :
    init.run() #한 번에 노드 초기화 (실제)
    result = f.eval()
    
print(result)

42


In [6]:
# 방법 4

sess = tf.InteractiveSession() #자동으로 기본 세션이 생성됨 (with블록 필요 X)
init.run()
result = f.eval()

print(result)

sess.close() #수동으로 닫아줘야 함 

42


### Tensorflow 프로그램은 일반적으로 두 부분으로 나뉨 
#### - construction phase : 계산 그래프 빌드 -- 일반적으로 ML 모델과 그것을 학습시키기 위해 필요한 계산을 나타내는 그래프 빌드 
#### - execution phase : 실행 -- 학습을 반복적으로 평가하는 루프를 실행하여 점진적으로 모델 파라미터 향상 

## Managing Graph

In [9]:
#아무런 설정이 없다면 노드는 기본적으로 default_graph(기본 그래프)에 생성됨 

x1 = tf.Variable(1)

x1.graph is tf.get_default_graph() #x1 그래프는 default_graph에서 가져올 수 있는가 

True

In [12]:
#그래프 설정 

graph = tf.Graph() #graph라는 이름의 텐서플로우 그래프 생성 

with graph.as_default(): #graph를 default로 노드 생성 
    x2 = tf.Variable(2)
    
x2.graph is graph 

True

In [14]:
x2.graph is tf.get_default_graph() #default_graph에서 가져올 수 없다 (존재하지 않음 )

False

#### 많은 중복 노드를 포함하는 기본 그래프가 생성될 경우 
- Jupyter 커널 재시작 
- tf,reset_default_graph() 실행  

## Lifecycle of a Node Value

### 노드 평가 시, 텐서플로우는 의존적인 노드 집합을 자동으로 결정하고 그 노드를 먼저 평가함 

In [7]:
W = tf.constant(3)#상수 설정 
x = W + 2
y = x + 5
z = x * 3

init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
init.run()

print(y.eval())
print(z.eval())

sess.close()

10
15


In [8]:
#위와 동일한 작업 (with구문 사용)

#그래프 빌드 
W = tf.constant(3)
x = W + 2
y = x + 5
z = x * 3

#실행 
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


- tensorflow는 자동으로 y는 x에 의존적이라는 것을 감지함 
    - y 평가 시 - 처음에 W 평가, 그 다음 x 평가, 그 다음 y 평가 후 y값 반환 
    - z 평가 시 - 처음에 W 평가, 그 다음 x 평가, 그 다음 z 평가 후 z값 반환 


- 이전 평가 결과를 다시 사용하지 않음
    - W와 x를 두 번 계산 

In [22]:
#W, x가 두 번 평가되지 않도록 

#그래프 빌드 
W = tf.constant(3)
x = W +2
y = x + 5
z = x * 3

#실행 
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()

init.run()
y_val, z_val = sess.run([y, z])

print(y_val)
print(z_val)

sess.close()

10
15


In [23]:
#위와 동일한 작업 (with구문 사용)

#그래프 빌드 
W = tf.constant(3)

x = W + 2
y = x + 5
z = x * 3

#실행 
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


## Linear Regression with Tensorflow

#### Tensorflow operations(ops)는 임의의 수의 입력값을 가져와서 여러개의 출력값을 생성할 수 있음 
### 입력과 출력은 tensor라고 하는 다차원 배열 
#### - Numpy 배열과 마찬가지로 tensor는 type과 shape을 가짐 
#### - 일반적으로 부동 소수점을 포함하지만 문자열을 전달할 때도 사용 가능 

In [9]:
#2장의 캘리포니아 주택 데이터 세트 사용하여 선형 회귀 수행 
#폐쇄방정식을 사용하여 최적의 세타값 구하기 

import numpy as np 
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

m, n = housing.data.shape
#모든 학습 데이터에 bias(X0=1)을 추가 
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data] #세타0은 늘 1

#그래프 빌드 
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)#전치
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y) #세타의 정ㅇ의 정규방정식(4장) 

#실행 
with tf.Session() as sess:
    theta_value = theta.eval()
    
print(theta_value)

[[ -3.74651413e+01]
 [  4.35734153e-01]
 [  9.33829229e-03]
 [ -1.06622010e-01]
 [  6.44106984e-01]
 [ -4.25131839e-06]
 [ -3.77322501e-03]
 [ -4.26648885e-01]
 [ -4.40514028e-01]]


## Implementing Gradient Descent

In [10]:
#경사하강법 (Batch Gradient Descent) 사용하여 최적의 세타값 구하기 

#Manually Computing the Gradients
#1. 수동으로 경사 계산

#정규화 --> Gradient Descent 사용 시, 정규화를 하지 않으면 학습이 느려질 수 있음 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

n_epochs = 1000 #주기 
learning_rate = 0.01

#그래프 빌드 
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta') 
#tf.random_uniorm(shape, 시작, 끝 범위) : 무작위 값을 포함하는 텐서 생성 
y_pred = tf.matmul(X, theta, name='prediction')

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error) #기울기계산 
#mse 미분(?)

training_op = tf.assign(theta, theta - learning_rate * gradients)
#assign() : 새로운 값을 변수에 할당 할 노드 생성, Batch Gradient Descent step 구현 
#theta에 저 연산 값을 넣는다 
#일종의 업데이트라 파악하자

#실행
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0: #100주기마다 출력
            print('Epoch', epoch, 'MSE = ', mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval() #mse가 최소가되는 세타값

print(best_theta)
#매 반복마다 MSE가 줄어듦 

Epoch 0 MSE =  4.55993
Epoch 100 MSE =  0.709479
Epoch 200 MSE =  0.631835
Epoch 300 MSE =  0.605129
Epoch 400 MSE =  0.585716
Epoch 500 MSE =  0.571203
Epoch 600 MSE =  0.5603
Epoch 700 MSE =  0.552076
Epoch 800 MSE =  0.545843
Epoch 900 MSE =  0.541097
[[ 2.06855249]
 [ 0.88613427]
 [ 0.15806651]
 [-0.31858847]
 [ 0.32595119]
 [ 0.00906561]
 [-0.04385704]
 [-0.56022257]
 [-0.53485698]]


In [11]:
#Using autodiff
#2. tf의 autodiff로 자동으로 gradient 계산 
#자동으로 효율적인 gradient 계산 가능 

n_epochs = 1000 #주기 
learning_rate = 0.01

#그래프 빌드 
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta') #tf.random_uniorm(shape, 시작, 끝 범위)
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

gradients =tf.gradients(mse, [theta])[0]
#tf.gradients() : op(여기서는 mse)와 변수 목록(theta)를 취하고 
                  #각 op에 대한 op의 기울기를 계산하기위해 하나의 변수 당 하나의 ops 목록을 만듦
                  #gradient 노드는 theta에 관한 mse의 gradient 벡터를 계산함
                  #tf.gradients(목적함수, 매개변수)

training_op = tf.assign(theta, theta - learning_rate * gradients)

#실행
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0: #100주기마다 출력
            print('Epoch', epoch, 'MSE = ', mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval() #mse가 최소가되는 세타값

print(best_theta)

Epoch 0 MSE =  5.6655
Epoch 100 MSE =  0.753876
Epoch 200 MSE =  0.637803
Epoch 300 MSE =  0.605272
Epoch 400 MSE =  0.582707
Epoch 500 MSE =  0.566445
Epoch 600 MSE =  0.554714
Epoch 700 MSE =  0.546251
Epoch 800 MSE =  0.540145
Epoch 900 MSE =  0.53574
[[ 2.06855226]
 [ 0.78212601]
 [ 0.145603  ]
 [-0.10891832]
 [ 0.14678352]
 [ 0.00588153]
 [-0.04058502]
 [-0.7487449 ]
 [-0.71063733]]


In [36]:
#Using an Optimizer
#3. tf의 gradient descent optimizer 사용하기  

n_epochs = 1000 #주기 
learning_rate = 0.01

#그래프 빌드 
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta') #tf.random_uniorm(shape, 시작, 끝 범위)
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse') 
gradients =tf.gradients(mse, [theta])[0]

#최적기 
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
 
training_op = optimizer.minimize(mse) #mse를 최소화하는것이 목적, 목적함수만 넘겨줌  
    
#실행
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0: #100주기마다 출력
            print('Epoch', epoch, 'MSE = ', mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval() #mse가 최소가되는 세타값

print(best_theta)

Epoch 0 MSE =  4.42048
Epoch 100 MSE =  0.671789
Epoch 200 MSE =  0.596169
Epoch 300 MSE =  0.577517
Epoch 400 MSE =  0.564692
Epoch 500 MSE =  0.555147
Epoch 600 MSE =  0.54798
Epoch 700 MSE =  0.542573
Epoch 800 MSE =  0.538476
Epoch 900 MSE =  0.535354
[[ 2.06855249]
 [ 0.87551254]
 [ 0.15063265]
 [-0.30869892]
 [ 0.32224935]
 [ 0.00649875]
 [-0.0430016 ]
 [-0.62438238]
 [-0.5982731 ]]


## Feeding Data to the Training Algorithm

In [42]:
#placeholder노드 : 실제로 계산을 수행하지 않음 - 런타임 시 출력하도록 지정한 데이터만 출력
# : 일반적으로 학습 중에 tensorflow에 학습 데이터를 전달하는 데에 사용 
A = tf.placeholder(dtype=tf.float32, shape=(None, 3)) #None = any size 
#A에 들어올 데이터는 2차원, _행3열의 크기어여야 함 
B = A + 5

with tf.Session() as sess :
    B_val_1 = B.eval(feed_dict={A:[[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A:[[4, 5, 6], [7, 8, 9]]})
    
print(B_val_1) #A에 1, 2, 3을 넣었을 때의 B평가
print("")
print(B_val_2)#A에 4, 5, 6 / 7, 8, 9를 넣었을 떄의 B평가 

[[ 6.  7.  8.]]

[[  9.  10.  11.]
 [ 12.  13.  14.]]


In [12]:
#Mini Batch Gradient 구현하기 

#그래프생성 
#placeholder노드 사용 
X = tf.placeholder(dtype=tf.float32, shape=(None, n+1), name='X')
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='y')

batch_size = 100 
n_batches = int(np.ceil(m / batch_size)) #batch의 전체 개수 

def fetch_batch(epoch, batch_index, bath_size):
    np.random.seed(epoch * n_batches +  batch_index)
    indices = np.random.randint(m, size=batch_size) #쪼개기
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X : X_batch, y: y_batch}) #batch를 하나씩 실행 
            
    best_theta = theta.eval()

In [47]:
print(best_theta)

[[ 2.06855249]
 [ 0.82963324]
 [ 0.11875443]
 [-0.26555082]
 [ 0.30571571]
 [-0.00450222]
 [-0.03932687]
 [-0.89985287]
 [-0.87050968]]


In [2]:
import numpy as np 
y = np.random.randint(100, size=10)
print(y)

[ 0 17 23 20 60 34 69 46  9 30]


## Saving and Restoring Models

- 모델 학습 후 파라미터를 디스크에 저장하여 필요할 때 불러와 사용 혹은 비교할 수 있도록 
- 학습 도중 일정한 간격으로 체크포인트를 저장하여 컴퓨터가 다운되었을 때, 마지막 체크포인트부터 진행될 수 있도록 

#### 1) 모든 변수 노드 생성 후 construction phase 마지막 부분에 Saver 노드 생성 
#### 2) execution phase에서 체크 포인트 파일의 세션과 경로를 전달하여 모델을 저장할 떄 마다 save() 메소드 호출 

In [17]:
n_epochs = 1000 #주기 
learning_rate = 0.01

#그래프 빌드 
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta') 
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error) #기울기계산 
training_op = tf.assign(theta, theta - learning_rate * gradients)

#실행
init = tf.global_variables_initializer()
#Saver 노드 
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0: #100주기마다 출력
            print('Epoch', epoch, 'MSE = ', mse.eval())
            save_path = saver.save(sess, '/Users/jsh/1 project/세미나;/tensor_tmp/my_model.cpkt')
        sess.run(training_op)
    
    best_theta = theta.eval() 
    save_path = saver.save(sess, '/Users/jsh/1 project/세미나;/tensor_tmp/my_model.cpkt')

print(best_theta)
#매 반복마다 MSE가 줄어듦 

Epoch 0 MSE =  6.26354
Epoch 100 MSE =  0.748867
Epoch 200 MSE =  0.571974
Epoch 300 MSE =  0.553418
Epoch 400 MSE =  0.545078
Epoch 500 MSE =  0.539356
Epoch 600 MSE =  0.535235
Epoch 700 MSE =  0.532252
Epoch 800 MSE =  0.530092
Epoch 900 MSE =  0.528524
[[ 2.06855249]
 [ 0.81631893]
 [ 0.13714455]
 [-0.20133014]
 [ 0.23542912]
 [ 0.00235965]
 [-0.04059832]
 [-0.77828342]
 [-0.74550068]]


In [19]:
#모델 재생성 
#construction phase 끝에 Saver노드 생성 후 
#exexution phase 시작부분에 init 노드를 이용해 변수 초기화 대신, Saver 객체의 restore()메소드 호출 
with tf.Session() as sess:
    saver.restore(sess, '/Users/jsh/1 project/세미나;/tensor_tmp/my_model.cpkt')
    best_theta_restored = theta.eval()
    
print(best_theta_restored)

INFO:tensorflow:Restoring parameters from /Users/jsh/1 project/세미나;/tensor_tmp/my_model.cpkt


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for /Users/jsh/1 project/세미나;/tensor_tmp/my_model.cpkt
	 [[Node: save_1/RestoreV2_1 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_1/Const_0_0, save_1/RestoreV2_1/tensor_names, save_1/RestoreV2_1/shape_and_slices)]]

Caused by op 'save_1/RestoreV2_1', defined at:
  File "C:\Users\jsh\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\jsh\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\jsh\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\jsh\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\jsh\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:

## Visualizing the Graph and Training Curves Using TensorBoard

#### 그래프에서 오류를 식별하고 병목 현상을 찾는 등의 작업에 유용 

#### 1) 텐서보드가 읽을 로그 디렉토리에 그래프 정의 및 학습통계를 기록 
- 프로그램을 실행할 때마다 다른 로그 디렉토리를 사용해야 함 (섞일 수가 있음)
    - 로그 디렉토리에 timestamp를 포함시키면 중복을 막을 수 있음 

In [4]:
#timestamp
#프로그램 시작 부분에 이 코드 추가 
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [5]:
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

NameError: name 'n' is not defined

In [6]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

NameError: name 'mse' is not defined

In [24]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [25]:
with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()
    # not shown
    
file_writer.close()

## Name Scopes 

#### 많은 노드들로 복잡해 지는 것을 막기 위해 이름 범위를 만들어 관련 노드를 그룹화 

In [26]:
#loss라는 이름 범위 내에서 error와 mse 정의 
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")
    
with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()
    # not shown
    
file_writer.close()

In [29]:
#범위 내에 정의 된 각 op의 앞에는 지정된 이름(loss)이 붙음 
#텐서보드에서는 loss안에 error와 mse노드가 숨겨져 있음 -- loss 노드의 플러스버튼을 누르면 확인 가능 
print(error.op.name)

loss/sub


In [30]:
print(mse.op.name)

loss/mse


## Modularity

- 두 개의 ReLU의 출력을 더하는 그래프 생성하기 
    - ReLU : 입력의 선형함수를 계산하고 양수이면 결과를 출력, 양수가 아니면 0을 출력 (후에 더 다룰 예정)

In [32]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

#가중치와 바이어스 
w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z2, 0., name="relu2")  

output = tf.add(relu1, relu2, name="output")

#이렇게 반복적인 코드는 유지가 어려우며 오류가 발생하기 쉬움 
#Tensorflow를 통해 DRY 상태를 유지할 수 있음 

In [33]:
#간단하게 ReLU를 만드는 함수 생성 
#5개의 ReLU를 만들고 합계를 출력하는 코드 
def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")
#add_n() : 텐서 목록의 합계를 계산하는 연산을 만듦 

#노드 생성 시 tensorflow는 해당 이름이 이미 있는지 확힌하고, 고유성을 위해 이름에 밑줄을 추가함 

In [34]:
#Name scope 사용 
def relu(X):
    with tf.name_scope("relu"):#변수들을 name scope 내로 이동 
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [35]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu2", tf.get_default_graph())
file_writer.close()

## Sharing Variables 

- 그래프의 다양한 컴포넌트 사이에서 변수를 공유하려면, 간단한 함수를 작성하고 필요로 하는 함수에 파라미터를 전달하면 됨 

